In [1]:
import os

In [2]:
pwd

'/Users/aksshar/PycharmProjects/Knee-Osteoarthritis-Severity-Grading/research'

In [3]:
os.chdir("../")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.cnnClassifier.constant import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config["artifacts_root"]])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config['data_ingestion']

        create_directories([config['root_dir']])

        data_ingestion_config= DataIngestionConfig(
            root_dir= config['root_dir'],
            source_URL= config['source_URL'],
            local_data_file= config['local_data_file'],
            unzip_dir= config['unzip_dir']
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-09 23:10:02,261: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-07-09 23:10:02,263: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-09 23:10:02,264: INFO: common: created directory at: artifacts]
[2023-07-09 23:10:02,264: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-09 23:10:02,698: INFO: 324913937: artifacts/data_ingestion/data.zip download! with following info: 
Content-Type: text/html; charset=utf-8
Cache-Control: no-cache, no-store, max-age=0, must-revalidate
Pragma: no-cache
Expires: Mon, 01 Jan 1990 00:00:00 GMT
Date: Sun, 09 Jul 2023 22:10:02 GMT
Strict-Transport-Security: max-age=31536000
Content-Security-Policy: script-src 'report-sample' 'nonce-piYz8ihPQF_Wbmx6GUTebA' 'unsafe-inline';object-src 'none';base-uri 'self';report-uri /_/DriveUntrustedContentHttp/cspreport;worker-src 'self'
Content-Security-Policy: require-trusted-types-for 'script';report-uri /_/DriveUntrustedContentHttp/cspreport
Permiss

BadZipFile: File is not a zip file